In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import LinkGCNConv
from torch_geometric.nn import SAGEConv
import torch.nn as nn
from pprint import pprint

import networkx as nx
import time
from torch_geometric.data import DataLoader
import importlib
from torch_geometric.data import Data
import pickle
import numpy as np


class MyOwnDataset2():
    def __init__(self,  root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #print(self.data)
        #print(self.data.edge_index)
        #print(self.data.num_features)
        self.num_features = self.data.num_features
        self.num_classes = 1
        self.filename = name
        
        # prepare edge_neighbors
        edges_dict = {}
        i=0
        for edge in self.data.edge_index[0]:
            edges_dict[i]= (self.data.edge_index[0][i],
                            self.data.edge_index[1][i])
            i+=1
            
        #print("\n edges_dict:")
        #pprint(edges_dict)
        #print("\n")
        """
            {
            0 : (1,3),
            1 : (1,4),
            2 : (2,0)
            ...
            }
            
            edge_neighbors= [[],[]]
            for edge in edges_dict.keys():
                for node in edges_dict[edge]:
                    for edge2 in edges_dict.keys():
                        if edge2 != edge and \
                           ( edges_dict[edge2][0] == node or \
                             edges_dict[edge2][1] == node ):
                             edge_neighbors[0].append(edge)
                             edge_neighbors[1].append(edge2)
                             
        """
        edge_neighbors= [[],[]]
        for edge in edges_dict.keys():
            for node in edges_dict[edge]:
                for edge2 in edges_dict.keys():
                    if edge2 != edge and ( edges_dict[edge2][0] == node or edges_dict[edge2][1] == node ):
                        edge_neighbors[0].append(edge)
                        edge_neighbors[1].append(edge2)
        
        self.data.edge_neighbors = torch.LongTensor(edge_neighbors)
        
        #print()
        #print("edge_neighbors")
        #pprint(self.data.edge_neighbors)
        #print()
        #print(type(self.data.edge_neighbors))
        #print()
        
        f.close()

class Net1(torch.nn.Module):
    def __init__(self, d1=16):
        super(Net1, self).__init__()
        self.conv1 = LinkGCNConv(1, d1)
        self.conv2 = LinkGCNConv(d1, 1)
        self.d1 = d1
        

    def forward(self, data):
        x, edge_neighbors = data.x, data.edge_neighbors

        x = self.conv1(x, edge_neighbors)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_neighbors)

        # output as multiclass target
        #return F.log_softmax(x, dim=1)
        
        # output as regression target
        return x
    
    def __str__(self):
        return "Net1-gcn(%d,%d)-gcn(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                               dataset.num_classes)
    
    
class Net2(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, d1 )
        #self.conv2 = GCNConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    
    def __str__(self):
        return "Net2-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)

    
    
class Net3(torch.nn.Module):
    def __init__(self, d1=300,d2=100):
        super(Net3, self).__init__()
        self.conv1 = SAGEConv(dataset.num_features, d1 )
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, dataset.num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    def __str__(self):
        return "Net3-SAGEgcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        dataset.num_features)

    


def loadDataset(collection, name=None):
    try:
        # import datasets
        themodule = importlib.import_module("torch_geometric.datasets")
        # get the function corresponding to collection
        method_to_call = getattr(themodule, collection)
        if name:
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except:
        # custom module
        method_to_call = globals()[collection]
       
        if name:
            
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
        


def transformMask(mask):
    train_mask = []
    i = 0
    for pick in mask:
        if pick[0]==1:
            train_mask.append(i)
        i+=1
    return train_mask


def shuffleTrainTestMasks(data, trainpct = 0.7):
    ysize = list(data.y.size())[0]
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[int(ysize*trainpct):] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
  

def shuffleTrainTestValMasks(data, trainpct = 0.7, valpct = 0.2):

    ysize = list(data.y.size())[0]
    #print("total ", ysize)
    #print(" train ",int(ysize*trainpct)-int(ysize*trainpct*valpct))
    #print(" val ",int(ysize*trainpct*valpct))
    #print(" test ",int(ysize*(1- trainpct) ))
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[:int(ysize*trainpct)] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    #print(" train sum ",data.train_mask.sum())
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    #print(" test sum ",data.test_mask.sum())
    
    # transform to list of indexes
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
    
    data.val_mask = data.train_mask[:int(ysize*trainpct*valpct)]
    data.train_mask = data.train_mask[int(ysize*trainpct*valpct):]

    
    #print(data.train_mask)
    #print(data.val_mask)
    #print(data.test_mask)
    
    

def trainTestEval(dataset, epochs=1, batch_size=32):
    global Net
    loader = DataLoader(dataset,  shuffle=False)
    i = 0
    #print(loader)
    #print(dir(loader))
    
    G = dataset.data
    print(G)
    start = time.time()


    # 1.  prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print("using ",device)
    model = Net.to(device)  
    data = G.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    # 2.  create a train_mask, and a test_mask (val_mask for further experiments)
    #shuffleTrainTestMasks(data)
    #shuffleTrainTestValMasks(data)
    shuffleTrainTestMasks(data)

    # 3. train some epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 25 == 0 :
            print("epoch-loss: ",epoch, loss)

    # 4. Model evaluation
    model.eval()
    #  classification in a multiclass setting
    #_, pred = model(data).max(dim=1)
    #correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    #acc = correct / data.test_mask.sum().item()
    #print('Accuracy: {:.4f}'.format(acc))


    # regression 
    pred = model(data)
    #print("target: ",data.y[data.test_mask])
    #print("prediction: ",pred[data.test_mask])
    #print(pred[data.test_mask].type())
    #print(data.y[data.test_mask].type())
    
    # prepare the normalized mean root squared error
    t = data.y[data.test_mask]
    y = pred[data.test_mask]
    nrmse = torch.sum((t - y) ** 2)/len(data.test_mask)
    nrmse = nrmse.sqrt()
    print("RMSE: ",nrmse)

    #m = torch.mean(t)
    #print("mean",m)
    #tmax = torch.max(t)
    #tmin = torch.min(t)
    #sd = tmax-tmin
    #print("sd",sd)
    #nrmse = (nrmse - m)/sd
    #print("NRMSE:",nrmse)


    endtime = time.time()
    print("Total train-test time: "+str(endtime-start))
    
    with open("results.txt","a") as f:
        #print(dir(dataset))
        f.write("\n")
        f.write(str(model)+" " 
                +str(dataset.filename)+" "  
                +"nrmse: "+str(nrmse.item())+" " 
                +"total time: "+str(endtime-start) 
                +" negative vals?: "+str(False) 
                +"\n"
               )
    
    del model

    #i+=1
    #if i==1:
    #    break

In [2]:
#dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_10_0_10_nb.pickle')
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
#print(dir(dataset.data))
#print()
global Net
Net=Net1(d1=5)
trainTestEval(dataset,  epochs=1)
del Net

Data(edge_index=[2, 7], x=[7, 1], y=[7])
tensor([[0, 0, 1, 1, 1, 2, 3],
        [1, 2, 2, 3, 4, 4, 4]])
Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.9520, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.9339, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 1.9290800094604492


In [16]:
#dataset = loadDataset(collection='MyOwnDataset2', name='precomputed/er_10_0_10_nb.pickle')
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
#print(dir(dataset.data))
#print()
global Net
Net=Net1(d1=55)
trainTestEval(dataset,  epochs=200)
del Net

Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.3592, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0284, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(0.0017, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(0.0013, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(0.0012, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(0.0014, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  150 tensor(0.0008, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  175 tensor(0.0010, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.1227, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 0.541536808013916


In [17]:
dataset = MyOwnDataset2(
    root='', 
    name='precomputed/er_5_0_45_eb.pickle')
global Net
Net=Net1(d1=55)
trainTestEval(dataset,  epochs=200)
del Net

# RMSE is between 15% adn 8% in a transductive setting

Data(edge_index=[2, 7], edge_neighbors=[2, 28], x=[7, 1], y=[7])
epoch-loss:  0 tensor(0.0720, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  25 tensor(0.0050, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  50 tensor(0.0048, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  75 tensor(0.0034, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  100 tensor(0.0033, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  125 tensor(0.0023, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  150 tensor(0.0024, device='cuda:0', grad_fn=<MseLossBackward>)
epoch-loss:  175 tensor(0.0024, device='cuda:0', grad_fn=<MseLossBackward>)
RMSE:  tensor(0.0835, device='cuda:0', grad_fn=<SqrtBackward>)
Total train-test time: 0.5560200214385986
